In [1]:
import gradio as gr
import random

In [2]:
# List of words to choose from
words = ["python", "hangman", "programming", "computer", "algorithm", "developer", "software"]

In [3]:
game_state = {
    "word": random.choice(words),
    "max_attempts": 6,
    "attempts": 0,
    "guessed_letters": set(),
    "current_word": []
}

In [4]:
# Reset function
def reset_game():
    game_state["word"] = random.choice(words)
    game_state["attempts"] = 0
    game_state["guessed_letters"] = set()
    game_state["current_word"] = ["_"] * len(game_state["word"])
    return update_display("", reset=True)

In [9]:
def update_display(guess, reset=False):
    if reset:
        return " ".join(game_state["current_word"]), "Game reset! Start guessing!", game_state["max_attempts"]

    guess = guess.lower().strip()

    # Input validation
    if len(guess) != 1 or not guess.isalpha():
        return " ".join(game_state["current_word"]), "Please enter a single valid letter.", game_state["max_attempts"] - game_state["attempts"]

    # Check if the letter has already been guessed
    if guess in game_state["guessed_letters"]:
        return " ".join(game_state["current_word"]), "You've already guessed that letter!", game_state["max_attempts"] - game_state["attempts"]

    # Add to guessed letters
    game_state["guessed_letters"].add(guess)

    # Check if the guess is in the word
    if guess in game_state["word"]:
        for i in range(len(game_state["word"])):
            if game_state["word"][i] == guess:
                game_state["current_word"][i] = guess
        message = f"Good job! '{guess}' is in the word."
    else:
        game_state["attempts"] += 1
        message = f"Sorry, '{guess}' is not in the word. {game_state['max_attempts'] - game_state['attempts']} attempts left"

    # Check if the game is over
    if "_" not in game_state["current_word"]:
        return " ".join(game_state["current_word"]), f"Congratulations! You guessed the word: {game_state['word']}", 0
    if game_state["attempts"] == game_state["max_attempts"]:
        return " ".join(game_state["current_word"]), f"Game over! The word was: {game_state['word']}", 0

    return " ".join(game_state["current_word"]), message, game_state["max_attempts"] - game_state["attempts"]

In [10]:
# Initial state setup
game_state["current_word"] = ["_"] * len(game_state["word"])

In [11]:
# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 🏆 Hangman Game")
    gr.Markdown("Try to guess the word by entering one letter at a time.")

    with gr.Row():
        word_display = gr.Textbox(value=" ".join(game_state["current_word"]), label="Current Word", interactive=False)
        attempts_display = gr.Textbox(value=str(game_state["max_attempts"]), label="Attempts Left", interactive=False)

    guess_input = gr.Textbox(label="Enter a letter", max_length=1)
    message_output = gr.Textbox(label="Message", interactive=False)

    guess_button = gr.Button("Submit Guess")
    reset_button = gr.Button("Reset Game")

    guess_button.click(update_display, inputs=[guess_input], outputs=[word_display, message_output, attempts_display])
    reset_button.click(reset_game, inputs=[], outputs=[word_display, message_output, attempts_display])

In [14]:
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7873
* Running on public URL: https://08049f7571f3b7be30.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
demo.close()

Closing server running on port: 7873
